In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [17]:
import pandas as pd
import numpy as np
import torch

import src.preprocessing as pre
import plotly.graph_objects as go
import src.utils as utils

set production plan

In [3]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')


In [4]:
production_plan = pre.decode(meta,mappings)[['Year','WeekTransplanted','Ranch','Class','Type','Variety','Ha']]

production_plan.reset_index(drop=True)

,Year,WeekTransplanted,Ranch,Class,Type,Variety,Ha
0,2024,4,VAP,BSUF,Uva Roja,King,0.8280
1,2024,4,VAP,BSUF,Uva Roja,Prince,0.4416
2,2024,4,VAP,CHE,Cherry Rojo,Tymoty,0.4224
3,2024,6,VAP,BSUF,Uva Roja,King,0.4500
4,2024,6,VAP,CHE,Cherry Rojo,Tymoty,0.4800
...,...,...,...,...,...,...,...
134,2024,29,VAP,CHE,Cherry Rojo,Tymoty,0.1764
135,2024,30,VAP,CHE,Cherry Naranja,Top 2417,0.1805
136,2024,31,SGB,BSUF,Uva Roja,King,0.5100
137,2024,31,SGB,BSUF,Uva Roja,King,0.0306


create summary by class/type, summary by ranch/class/type

In [5]:
summary_ranch = production_plan.groupby(['Ranch','Class','Type']).agg(
    {'Ha':'sum',
     'WeekTransplanted':['min','max','count']
})

summary_class = production_plan.groupby(['Class','Type']).agg(
    {'Ha':'sum',
     'WeekTransplanted':['min','max','nunique'],
     'Ranch':'nunique'
})

summary_class_ranch = production_plan.groupby(['Class','Type','Ranch']).agg(
    {'Ha':'sum',
     'WeekTransplanted':['min','max','count']
})

In [6]:
summary_ranch

Ha WeekTransplanted          
                                sum              min max count
Ranch Class Type                                              
OAP   BSUF  Uva Amarilla     0.7200               17  26     3
            Uva Roja         7.4880               15  26    12
      CHE   Cherry Rojo      3.9920               15  27     7
            Cherry Rosa      0.6660               15  22     3
            Pera Amarilla    0.1440               20  20     2
SFB   BSUF  Uva Amarilla     0.3200               14  28     2
            Uva Roja         1.3620               14  28     3
      CHE   Cherry Naranja   0.0172               14  14     1
            Cherry Rojo      1.1160               14  28     2
            Cherry Rosa      0.3040               14  28     2
            Pera Amarilla    0.0960               14  14     1
            Pera Roja        0.0960               14  14     1
SGB   BSUF  Uva Amarilla     0.1715               15  17     2
            Uva Roja         1.5028               12  31     4
      CHE   Cherry Naranja   0.1078               15  17     2
            Cherry Rojo      1.0251               12  31     3
            Cherry Rosa      0.2632               15  17     2
            Pera Amarilla    0.0595               17  17     1
            Pera Roja        0.0595               17  17     1
SJB   BSUF  Uva Amarilla     0.1900               14  24     2
            Uva Roja         1.9995               14  24     3
      CHE   Cherry Naranja   0.0804               14  24     2
            Cherry Rojo      1.2102               14  24     2
            Cherry Rosa      0.2994               14  24     2
            Pera Amarilla    0.1092               14  24     2
            Pera Roja        0.1002               14  24     2
SMB   BSUF  Uva Amarilla     0.2400               18  21     2
            Uva Roja         2.1800               16  22     6
      CHE   Cherry Naranja   0.0600               18  18     1
            Cherry Rojo      1.9000               16  27     3
            Cherry Rosa      0.2800               18  21     2
            Pera Amarilla    0.0800               18  18     1
VAP   BSUF  Uva Amarilla     0.8330                6  27     3
            Uva Roja        13.3812                4  29    26
      CHE   Cherry Naranja   0.3875               12  30     4
            Cherry Rojo      8.9636                4  29    13
            Cherry Rosa      1.9713                9  29     6
            Pera Amarilla    0.1150               16  16     1
            Pera Roja        0.2050               11  16     2

In [7]:
summary_class

Ha WeekTransplanted               Ranch
                          sum              min max nunique nunique
Class Type                                                        
BSUF  Uva Amarilla     2.4745                6  28      12       6
      Uva Roja        27.9135                4  31      19       6
CHE   Cherry Naranja   0.6529               12  30       9       5
      Cherry Rojo     18.2069                4  31      15       6
      Cherry Rosa      3.7839                9  29      13       6
      Pera Amarilla    0.6037               14  24       6       6
      Pera Roja        0.4607               11  24       5       4

In [8]:
summary_class_ranch

Ha WeekTransplanted          
                                sum              min max count
Class Type           Ranch                                    
BSUF  Uva Amarilla   OAP     0.7200               17  26     3
                     SFB     0.3200               14  28     2
                     SGB     0.1715               15  17     2
                     SJB     0.1900               14  24     2
                     SMB     0.2400               18  21     2
                     VAP     0.8330                6  27     3
      Uva Roja       OAP     7.4880               15  26    12
                     SFB     1.3620               14  28     3
                     SGB     1.5028               12  31     4
                     SJB     1.9995               14  24     3
                     SMB     2.1800               16  22     6
                     VAP    13.3812                4  29    26
CHE   Cherry Naranja SFB     0.0172               14  14     1
                     SGB     0.1078               15  17     2
                     SJB     0.0804               14  24     2
                     SMB     0.0600               18  18     1
                     VAP     0.3875               12  30     4
      Cherry Rojo    OAP     3.9920               15  27     7
                     SFB     1.1160               14  28     2
                     SGB     1.0251               12  31     3
                     SJB     1.2102               14  24     2
                     SMB     1.9000               16  27     3
                     VAP     8.9636                4  29    13
      Cherry Rosa    OAP     0.6660               15  22     3
                     SFB     0.3040               14  28     2
                     SGB     0.2632               15  17     2
                     SJB     0.2994               14  24     2
                     SMB     0.2800               18  21     2
                     VAP     1.9713                9  29     6
      Pera Amarilla  OAP     0.1440               20  20     2
                     SFB     0.0960               14  14     1
                     SGB     0.0595               17  17     1
                     SJB     0.1092               14  24     2
                     SMB     0.0800               18  18     1
                     VAP     0.1150               16  16     1
      Pera Roja      SFB     0.0960               14  14     1
                     SGB     0.0595               17  17     1
                     SJB     0.1002               14  24     2
                     VAP     0.2050               11  16     2

create planting plans

In [9]:
ranch_pivot = production_plan.pivot_table(
    index=['Ranch', 'Class', 'Type'],
    columns='WeekTransplanted',
    values='Ha',
    aggfunc='sum',
    fill_value=0,
)

class_pivot = production_plan.pivot_table(
    index=['Class', 'Type'],
    columns='WeekTransplanted',
    values='Ha',
    aggfunc='sum',
    fill_value=0,
)
class_pivot


WeekTransplanted          4      6       9       11      12      14      15  \
Class Type                                                                    
BSUF  Uva Amarilla    0.0000  0.176  0.0000  0.0000  0.0000  0.2500  0.0525   
      Uva Roja        1.2696  1.854  0.9832  1.9049  1.2856  1.5019  1.2060   
CHE   Cherry Naranja  0.0000  0.000  0.0000  0.0000  0.0640  0.0532  0.0840   
      Cherry Rojo     0.4224  1.520  1.2060  0.0000  1.5294  1.3942  0.5940   
      Cherry Rosa     0.0000  0.000  0.2464  0.0000  0.1920  0.3680  0.3660   
      Pera Amarilla   0.0000  0.000  0.0000  0.0000  0.0000  0.1500  0.0000   
      Pera Roja       0.0000  0.000  0.0000  0.0900  0.0000  0.1410  0.0000   

WeekTransplanted         16      17    18     20    21     22      24     26  \
Class Type                                                                     
BSUF  Uva Amarilla    0.207  0.2630  0.12  0.234  0.12  0.000  0.1000  0.342   
      Uva Roja        1.942  1.6942  0.56  1.530  1.00  1.172  1.9716  1.386   
CHE   Cherry Naranja  0.023  0.0238  0.06  0.000  0.00  0.000  0.0444  0.000   
      Cherry Rojo     1.929  1.4159  0.00  1.044  0.00  1.458  0.5520  0.000   
      Cherry Rosa     0.138  0.3983  0.12  0.000  0.16  0.234  0.1554  0.000   
      Pera Amarilla   0.115  0.0595  0.08  0.144  0.00  0.000  0.0552  0.000   
      Pera Roja       0.115  0.0595  0.00  0.000  0.00  0.000  0.0552  0.000   

WeekTransplanted          27    28      29      30      31  
Class Type                                                  
BSUF  Uva Amarilla    0.4500  0.16  0.0000  0.0000  0.0000  
      Uva Roja        3.3405  0.76  2.0114  0.0000  0.5406  
CHE   Cherry Naranja  0.1200  0.00  0.0000  0.1805  0.0000  
      Cherry Rojo     4.1878  0.38  0.1764  0.0000  0.3978  
      Cherry Rosa     0.4500  0.08  0.8758  0.0000  0.0000  
      Pera Amarilla   0.0000  0.00  0.0000  0.0000  0.0000  
      Pera Roja       0.0000  0.00  0.0000  0.0000  0.0000

create graphs

In [10]:


# Prepare data for the stacked bar plot
data = []
for class_type in class_pivot.index:
    data.append(go.Bar(
        name=f'{class_type}',
        x=class_pivot.columns,
        y=class_pivot.loc[class_type]
    ))

# Create the figure
fig = go.Figure(data=data)

# Update layout for the plot
fig.update_layout(
    barmode='stack',
    title='Stacked Bar Plot of Hectares by Week Transplanted',
    xaxis_title='Week Transplanted',
    yaxis_title='Hectares (Ha)',
    legend_title='Class and Type',
    xaxis=dict(tickangle=45, categoryorder='total descending'),
    legend=dict(x=1.05, y=1)
)

# Show plot
fig.show()


In [11]:
# Prepare data for the stacked bar plot by Ranch
ranch_pivot = production_plan.pivot_table(
    index='Ranch',
    columns='WeekTransplanted',
    values='Ha',
    aggfunc='sum',
    fill_value=0,
)

# Prepare data for the stacked bar plot
ranch_data = []
for ranch in ranch_pivot.index:
    ranch_data.append(go.Bar(
        name=f'{ranch}',
        x=ranch_pivot.columns,
        y=ranch_pivot.loc[ranch]
    ))

# Create the figure
ranch_fig = go.Figure(data=ranch_data)

# Update layout for the plot
ranch_fig.update_layout(
    barmode='stack',
    title='Stacked Bar Plot of Hectares by Week Transplanted (Ranch)',
    xaxis_title='Week Transplanted',
    yaxis_title='Hectares (Ha)',
    legend_title='Ranch',
    xaxis=dict(tickangle=45, categoryorder='total descending'),
    legend=dict(x=1.05, y=1)
)

# Show plot
ranch_fig.show()


initialize model

In [12]:
model = utils.create_model(train,num_epochs=1)

Epoch [1/200], Loss: 4404239.7315


create predictions

In [38]:
preds = utils.predict_harvest(model,test)


In [44]:
production_plan['Kilos'] = preds.sum(axis=1)

historical and predicted yield

In [48]:


class_preds = production_plan.groupby(['Class','Type']).agg(
    {'Ha':'sum',
     'WeekTransplanted':['min','max','nunique'],
     'Ranch':'nunique',
     'Kilos':'sum'
})

class_ranch_preds = production_plan.groupby(['Class','Type','Ranch']).agg(
    {'Ha':'sum',
     'WeekTransplanted':['min','max','count'],
     'Kilos':'sum'
})
class_preds['Yield'] = class_preds['Kilos']/class_preds['Ha']
class_ranch_preds['Yield'] = class_ranch_preds['Kilos']/class_ranch_preds['Ha']

class_ranch_preds


Ha WeekTransplanted                   Kilos  \
                                sum              min max count           sum   
Class Type           Ranch                                                     
BSUF  Uva Amarilla   OAP     0.7200               17  26     3   3089.205078   
                     SFB     0.3200               14  28     2   2038.649414   
                     SGB     0.1715               15  17     2   2034.667969   
                     SJB     0.1900               14  24     2   2035.157227   
                     SMB     0.2400               18  21     2   2048.662354   
                     VAP     0.8330                6  27     3   3063.554688   
      Uva Roja       OAP     7.4880               15  26    12  12909.986328   
                     SFB     1.3620               14  28     3   3191.653809   
                     SGB     1.5028               12  31     4   4213.804199   
                     SJB     1.9995               14  24     3   3207.151855   
                     SMB     2.1800               16  22     6   6370.477051   
                     VAP    13.3812                4  29    26  27505.291016   
CHE   Cherry Naranja SFB     0.0172               14  14     1   1001.381836   
                     SGB     0.1078               15  17     2   1993.500977   
                     SJB     0.0804               14  24     2   1992.743042   
                     SMB     0.0600               18  18     1   1003.489258   
                     VAP     0.3875               12  30     4   3995.637207   
      Cherry Rojo    OAP     3.9920               15  27     7   7416.484375   
                     SFB     1.1160               14  28     2   2100.333496   
                     SGB     1.0251               12  31     3   3103.128174   
                     SJB     1.2102               14  24     2   2102.932861   
                     SMB     1.9000               16  27     3   3160.851562   
                     VAP     8.9636                4  29    13  13643.461914   
      Cherry Rosa    OAP     0.6660               15  22     3   3154.019775   
                     SFB     0.3040               14  28     2   2078.711914   
                     SGB     0.2632               15  17     2   2077.656494   
                     SJB     0.2994               14  24     2   2078.626465   
                     SMB     0.2800               18  21     2   2090.246094   
                     VAP     1.9713                9  29     6   6279.957031   
      Pera Amarilla  OAP     0.1440               20  20     2   2059.632080   
                     SFB     0.0960               14  14     1   1024.612793   
                     SGB     0.0595               17  17     1   1019.071472   
                     SJB     0.1092               14  24     2   2035.705566   
                     SMB     0.0800               18  18     1   1025.122559   
                     VAP     0.1150               16  16     1   1030.359497   
      Pera Roja      SFB     0.0960               14  14     1   1018.676636   
                     SGB     0.0595               17  17     1   1013.135315   
                     SJB     0.1002               14  24     2   2023.588623   
                     VAP     0.2050               11  16     2   2038.270508   

                                   Yield  
                                          
Class Type           Ranch                
BSUF  Uva Amarilla   OAP     4290.562609  
                     SFB     6370.779419  
                     SGB    11863.953171  
                     SJB    10711.353824  
                     SMB     8536.093140  
                     VAP     3677.736720  
      Uva Roja       OAP     1724.090055  
                     SFB     2343.358156  
                     SGB     2803.968725  
                     SJB     1603.976922  
                     SMB     2922.237179  
                     VAP     2055.517518  
CHE   Cherry Naranja SFB   